In [4]:
import clickhouse_connect
from datetime import date

master_client = clickhouse_connect.get_client(
    host='localhost',  
    port=8123,
    username='default',
    password='',
    database='default'
)

shard1_client = clickhouse_connect.get_client(host='localhost', port=8124)
shard2_client = clickhouse_connect.get_client(host='localhost', port=8125)


In [7]:
# CREATE: Создание таблицы
def create_tables():
    master_client.command('''
    CREATE TABLE IF NOT EXISTS `default`.users_local ON CLUSTER my_cluster (
        id UInt32,
        name String,
        age UInt8,
        date_registered Date
    ) ENGINE = ReplicatedMergeTree('/clickhouse/tables/{shard}/users_local', '{replica}')
    PARTITION BY toYYYYMM(date_registered)
    order by id;
    ''')


    master_client.command('''
    CREATE TABLE IF NOT EXISTS `default`.users_local_dist (
        id UInt32,
        name String,
        age UInt8,
        date_registered Date      
    ) ENGINE = Distributed(my_cluster, default, users_local, rand());
    ''')
    print("Tables created across cluster")

In [ ]:
# INSERT: Вставка данных
def insert_user(user_id, name, age, date_reg):
    data = [(user_id, name, age, date_reg)]
    master_client.insert('users_local_dist', data, column_names=['id', 'name', 'age', 'date_registered'])
    print(f"Inserted user {name}")


In [ ]:
# SELECT: Получение данных
def get_all_users():
    result = master_client.query('SELECT * FROM users_local_dist ORDER BY id')
    return result.result_rows

In [ ]:
# UPDATE: Обновление данных
def update_user_age(user_id, new_age):
    query = f'''
    ALTER TABLE users_local ON CLUSTER my_cluster
    UPDATE age = {new_age}
    WHERE id = {user_id}
    '''
    master_client.command(query)
    print(f"Updated age for user ID {user_id}")

In [17]:
# DELETE: Удаление данных
def delete_user(user_id):
    query = f'''
    ALTER TABLE users_local ON CLUSTER my_cluster
    DELETE WHERE id = {user_id}
    '''
    master_client.command(query)
    print(f"Deleted user ID {user_id}")

In [8]:
create_tables()

Tables created across cluster


In [ ]:
insert_user(1, 'Alice', 30, date(2025, 6, 1))
insert_user(2, 'Bob', 27, date(2025, 6, 2))
insert_user(3, 'Charlie', 22, date(2025, 5, 20))
insert_user(4, 'Diana', 35, date(2025, 4, 15))
insert_user(5, 'Ethan', 40, date(2025, 3, 5))
insert_user(6, 'Fiona', 29, date(2025, 6, 3))
insert_user(7, 'George', 31, date(2025, 2, 18))
insert_user(8, 'Hannah', 26, date(2025, 1, 12))
insert_user(9, 'Ivan', 38, date(2024, 12, 25))
insert_user(10, 'Julia', 24, date(2025, 6, 4))

In [19]:
get_all_users()

[(1, 'Alice', 30, datetime.date(2025, 6, 1)),
 (2, 'Bob', 29, datetime.date(2025, 6, 2)),
 (3, 'Charlie', 22, datetime.date(2025, 5, 20)),
 (4, 'Diana', 35, datetime.date(2025, 4, 15)),
 (5, 'Ethan', 40, datetime.date(2025, 3, 5)),
 (6, 'Fiona', 29, datetime.date(2025, 6, 3)),
 (7, 'George', 31, datetime.date(2025, 2, 18)),
 (8, 'Hannah', 26, datetime.date(2025, 1, 12)),
 (9, 'Ivan', 38, datetime.date(2024, 12, 25))]

In [15]:
update_user_age(2, 29)

Updated age for user ID 2


In [18]:
delete_user(10)

Deleted user ID 10
